### Data Loading & EDA

In [1]:
# Import neccesary libraries
import json
import pandas as pd
import numpy as np
from pathlib import Path
from collections import Counter
from bert_score import score
import re
import matplotlib.pyplot as plt
import seaborn as sns
import logging
from tqdm import tqdm
from transformers import pipeline

In [2]:
# 로깅 설정
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)

In [3]:
def analyze_categories(directory_path):
    """
    Analyzes the categories in the dataset and counts documents in each category.
    
    Args:
        directory_path (str): Path to the directory containing JSON files
    
    Returns:
        None: Prints the analysis results
    """
    path = Path(directory_path)
    json_files = list(path.glob('*.json'))
    print(f"The # of discovered JSON files: {len(json_files)}")
    
    if not json_files:
        print("No JSON files found in the directory.")
        return
    
    # Process each JSON file
    for file_idx, json_file in enumerate(json_files):
        print(f"\n--- Analyzing file {file_idx+1}/{len(json_files)}: {json_file.name} ---")
        
        with open(json_file, 'r', encoding='utf-8') as f:
            try:
                data = json.load(f)
                
                if 'documents' not in data:
                    print(f"No 'documents' field found in {json_file.name}")
                    continue
                
                total_docs = len(data['documents'])
                print(f"Total documents in this file: {total_docs}")
                
                # Count documents by category
                categories = [doc.get('category', 'Unknown') for doc in data['documents']]
                category_counts = Counter(categories)
                
                # Print category distribution
                print("\nCategory distribution:")
                print("-" * 40)
                print(f"{'Category':<20} | {'Count':<10} | {'Percentage':<10}")
                print("-" * 40)
                
                for category, count in sorted(category_counts.items(), key=lambda x: x[1], reverse=True):
                    percentage = (count / total_docs) * 100
                    print(f"{category:<20} | {count:<10} | {percentage:.2f}%")
                
                # Additional statistics for a more detailed view
                # Check subcategories (media_sub_type) if needed
                print("\nTop 5 media subtypes:")
                media_subtypes = [doc.get('media_sub_type', 'Unknown') for doc in data['documents']]
                subtype_counts = Counter(media_subtypes).most_common(5)
                for subtype, count in subtype_counts:
                    print(f"- {subtype}: {count} documents")
                
            except json.JSONDecodeError:
                print(f"Error: Could not parse {json_file.name} as valid JSON")
                continue
            except Exception as e:
                print(f"Error processing {json_file.name}: {str(e)}")
                continue

In [4]:
def check_data_structure(directory_path):
    """The function which checks the internal structure of the dataset"""
    path = Path(directory_path)
    json_files = list(path.glob('*.json'))
    print(f"The # of discovered JSON files: {len(json_files)}")
    
    if json_files:
        with open(json_files[0], 'r', encoding='utf-8') as f:
            data = json.load(f)
            print("\nData type:", type(data))
            
            if 'documents' in data:
                # Only filter the economy category news
                econ_fin_docs = [doc for doc in data['documents'] 
                               if doc['category'] in ['경제']]
                
                print("\n=== Status of econ news data ===")
                print(f"The # of documents related to economics: {len(econ_fin_docs)}")
                
                if econ_fin_docs:
                    print("\nThe sample of the first econ news:")
                    # 제한 없이 전체 출력 ([:1000] 부분 제거)
                    print(json.dumps(econ_fin_docs[0], indent=2, ensure_ascii=False))
                    print("\nKey structure of such document:")
                    print(list(econ_fin_docs[0].keys()))

In [5]:
# File path
news_train_file_path = "C:/Users/99kih/OneDrive/바탕 화면/논문 스터디/Project/Data/Training/신문기사_train_original"
news_val_file_path = "C:/Users/99kih/OneDrive/바탕 화면/논문 스터디/Project/Data/Validation/신문기사_valid_original"

In [6]:
# Check the # of categories of the dataset
print(analyze_categories(news_train_file_path))
print(analyze_categories(news_val_file_path))

The # of discovered JSON files: 1

--- Analyzing file 1/1: train_original.json ---
Total documents in this file: 243983

Category distribution:
----------------------------------------
Category             | Count      | Percentage
----------------------------------------
종합                   | 177558     | 72.77%
경제                   | 23938      | 9.81%
사회                   | 17650      | 7.23%
정치                   | 16389      | 6.72%
스포츠                  | 5174       | 2.12%
IT,과학                | 1931       | 0.79%
교육/입시/NIE            | 1190       | 0.49%
부동산                  | 74         | 0.03%
보건/의료                | 55         | 0.02%
기업                   | 17         | 0.01%
북한/한반도정세             | 5          | 0.00%
선거                   | 2          | 0.00%

Top 5 media subtypes:
- 지역지: 169724 documents
- 경제지: 68388 documents
- 전문지: 5871 documents
None
The # of discovered JSON files: 1

--- Analyzing file 1/1: valid_original.json ---
Total documents in this file: 30122

Categ

In [7]:
# Check the sturcture of the train / val news text
print(check_data_structure(news_train_file_path))
print(check_data_structure(news_val_file_path))

The # of discovered JSON files: 1

Data type: <class 'dict'>

=== Status of econ news data ===
The # of documents related to economics: 23938

The sample of the first econ news:
{
  "id": "291514704",
  "category": "경제",
  "media_type": "online",
  "media_sub_type": "지역지",
  "media_name": "광양신문",
  "size": "small",
  "char_count": "886",
  "publish_date": "2018-01-12 18:42:06",
  "title": "여수광양항만공사, 광양항 배후단지‘투자유치’시동",
  "text": [
    [
      {
        "index": 0,
        "sentence": "중국 투자의향기업 직접 방문·투자협약 체결 협의",
        "highlight_indices": "10,12"
      }
    ],
    [
      {
        "index": 1,
        "sentence": "이성훈 sinawi@hanmail.net",
        "highlight_indices": ""
      }
    ],
    [
      {
        "index": 2,
        "sentence": "여수광양항만공사(사장 방희석)는 14일부터 20일까지 광양항 배후단지 투자 유치를 위해 CEO가 직접 참여하는 투자유치 활동을 중국 중남부지역에서 펼친다.",
        "highlight_indices": "36,38;54,56"
      }
    ],
    [
      {
        "index": 3,
        "sentence": "광양만권경제자유구역청과 합동으로 진행되는 이번 투자유치활동은 방희석 사장이 직접 중

### Reconstructing JSON file

In [8]:
# BERTScore 계산을 위한 함수
def compute_bert_score(text1, text2):
    """
    두 텍스트 간의 BERTScore를 계산합니다.
    
    Args:
        text1 (str): 첫 번째 텍스트
        text2 (str): 두 번째 텍스트
    
    Returns:
        float: 두 텍스트 간의 BERTScore (F1 점수)
    """
    
    # BERTScore 계산 (한국어 모델 사용)
    P, R, F1 = score([text1], [text2], lang="ko")
    
    # F1 점수 반환 (일반적으로 가장 많이 사용되는 지표)
    return F1.item()

In [9]:
# SHAP을 통한 중요 토큰 식별 함수 (실제 구현 시 추가)
def identify_important_tokens_with_shap(text, summary):
    """
    SHAP을 사용하여 중요한 토큰을 식별합니다. (향후 구현 예정)
    
    Args:
        text (str): 원문 텍스트
        summary (str): 요약 텍스트
    
    Returns:
        dict: {토큰: 중요도} 형태의 딕셔너리
    """
    # 임시로 빈 함수 - 실제 SHAP 구현은 향후 추가 예정
    # 임시 데이터 반환
    tokens = text.split()[:10]  # 임시로 처음 10개 단어만 사용
    importance_scores = np.random.random(len(tokens))
    important_tokens = {tokens[i]: float(importance_scores[i]) for i in range(len(tokens))}
    return important_tokens

In [10]:
# LLM을 통한 요약문 생성 함수 (실제 구현 시 추가)
def generate_summary_with_llm(text, annotations=None):
    """
    LLM을 사용하여 텍스트 요약을 생성합니다. (향후 구현 예정)
    
    Args:
        text (str): 원문 텍스트
        annotations (dict, optional): 중요 토큰 및 주석
    
    Returns:
        str: 생성된 요약문
    """
    # 실제 구현 시 적절한 LLM API 사용
    # 임시로 원문의 앞부분을 반환
    if text:
        shortened_text = ' '.join(text.split()[:30])
        return f"이것은 임시 요약입니다: {shortened_text}..."
    return "요약 내용 없음"

In [11]:
def clean_text(text):
    """
    텍스트를 정제하는 함수: 기자 이름, 이메일, 불필요한 특수문자 등을 제거합니다.
    
    Args:
        text (str): 정제할 원본 텍스트
    
    Returns:
        str: 정제된 텍스트
    """
    # 1. 이메일 주소 제거
    text = re.sub(r'\S+@\S+\.\S+', '', text)
    
    # 2. '기자' 패턴 제거 (예: "홍길동 기자", "김기자")
    text = re.sub(r'\S+\s+기자', '', text)
    text = re.sub(r'\S+기자', '', text)
    
    # 3. 특정 패턴 제거
    patterns_to_remove = [
        r'사진제공.+',           # '사진제공' 다음에 오는 텍스트
        r'사진=.+?기자',         # '사진=' 패턴
        r'자료제공.+',           # '자료제공' 다음에 오는 텍스트
        r'\(서울=.+?\)',         # (서울=...) 패턴
        r'\([가-힣]{2,}=.+?\)',  # (지역명=...) 패턴
        r'\[.+?\]',              # [...] 대괄호 안의 내용
        r'▲.+',                  # ▲ 다음에 오는 텍스트
        r'■.+',                  # ■ 다음에 오는 텍스트
        r'◆.+',                  # ◆ 다음에 오는 텍스트
        r'【.+】',               # 【...】 패턴
        r'『.+』',               # 『...』 패턴
    ]
    
    for pattern in patterns_to_remove:
        text = re.sub(pattern, '', text)
    
    # 4. 기타 불필요한 특수문자 제거 (단, 인용문에 필요한 따옴표와 기본 구둣점은 유지)
    text = re.sub(r'[^\w\s\.\,\"\'\?\!\:\;]+', ' ', text)  # 기본 문장부호 외 특수문자 제거
    
    # 5. 연속된 공백 제거
    text = re.sub(r'\s+', ' ', text)
    
    # 6. 문장 앞뒤 공백 제거
    text = text.strip()
    
    return text

In [16]:
def process_news_dataset(input_path, output_path, limit=None):
    """
    뉴스 데이터셋을 처리하여 새로운 JSON 구조로 변환합니다.
    
    Args:
        input_path (str): 입력 데이터셋 경로
        output_path (str): 출력 JSON 파일 경로
        limit (int, optional): 처리할 최대 문서 수 (테스트용)
    """
    path = Path(input_path)
    json_files = list(path.glob('*.json'))
    print(f"발견된 JSON 파일 수: {len(json_files)}")
    
    if not json_files:
        print("디렉토리에 JSON 파일이 없습니다.")
        return
    
    processed_data = {
        "documents": []
    }
    
    doc_count = 0
    
    # 각 JSON 파일 처리
    for file_idx, json_file in enumerate(tqdm(json_files, desc="파일 처리 중")):
        with open(json_file, 'r', encoding='utf-8') as f:
            try:
                data = json.load(f)
                
                if 'documents' not in data:
                    print(f"{json_file.name}에서 'documents' 필드를 찾을 수 없습니다.")
                    continue
                
                # 경제 카테고리 문서만 필터링
                econ_docs = [doc for doc in data['documents'] if doc.get('category') == '경제']
                
                for doc in tqdm(econ_docs, desc=f"파일 {file_idx+1}/{len(json_files)} 문서 처리 중", leave=False):
                    # 제한된 수의 문서만 처리 (테스트용)
                    if limit and doc_count >= limit:
                        break
                    
                    # 원문 추출
                    if 'text' not in doc:
                        continue
                    
                    text_blocks = doc['text']
                    original_text = ""
                    
                    # 문장 구성
                    for block in text_blocks:
                        for sentence in block:
                            original_text += sentence['sentence'] + " "
                    
                    # 텍스트 정제
                    # original_text = clean_text(original_text)
                    
                    # Ground Truth 요약문 (추상 요약)
                    if 'abstractive' in doc and doc['abstractive']:
                        ground_truth = doc['abstractive'][0] if isinstance(doc['abstractive'], list) else doc['abstractive']
                    else:
                        # 추상 요약이 없는 경우 건너뛰기
                        continue
                    
                    # 이 부분들은 향후에 실제 구현 예정 (현재는 구조만)
                    # 1. LLM을 사용한 첫 번째 요약문 생성 (원문만 사용)
                    summary_without_annotations = generate_summary_with_llm(original_text)
                    
                    # 2. SHAP을 통한 중요 토큰 식별
                    important_tokens = identify_important_tokens_with_shap(original_text, ground_truth)
                    
                    # 3. 주석 생성 (임시)
                    annotations = {}
                    for token, importance in important_tokens.items():
                        annotations[token] = f"이 단어는 요약에 중요합니다. 중요도: {importance:.4f}"
                    
                    # 4. 두 번째 요약문 생성 (원문 + 주석 사용)
                    summary_with_annotations = generate_summary_with_llm(original_text, annotations)
                    
                    # 5. BERTScore 계산 (임시)
                    bert_score_original = compute_bert_score(original_text, ground_truth)
                    bert_score_summary1 = compute_bert_score(summary_without_annotations, ground_truth)
                    bert_score_summary2 = compute_bert_score(summary_with_annotations, ground_truth)
                    
                    # 새로운 구조의 문서 생성
                    processed_doc = {
                        "id": doc.get('id', ''),
                        "category": doc.get('category', ''),
                        "title": doc.get('title', ''),
                        "original_text": original_text,
                        "annotations": annotations,
                        "ground_truth_summary": ground_truth,
                        "summary_without_annotations": summary_without_annotations,
                        "summary_with_annotations": summary_with_annotations,
                        "bert_score_original": float(bert_score_original),
                        "bert_score_summary_without_annotations": float(bert_score_summary1),
                        "bert_score_summary_with_annotations": float(bert_score_summary2)
                    }
                    
                    processed_data["documents"].append(processed_doc)
                    doc_count += 1
                    
                    # 제한된 수의 문서만 처리 (테스트용)
                    if limit and doc_count >= limit:
                        break
                    
            except json.JSONDecodeError:
                print(f"오류: {json_file.name}을 유효한 JSON으로 파싱할 수 없습니다.")
                continue
            except Exception as e:
                print(f"{json_file.name} 처리 중 오류 발생: {str(e)}")
                continue
        
        # 제한된 수의 문서만 처리 (테스트용)
        if limit and doc_count >= limit:
            break
    
    # 처리된 데이터에 메타데이터 추가
    processed_data["metadata"] = {
        "total_documents": len(processed_data["documents"]),
        "source_files": len(json_files),
        "creation_date": pd.Timestamp.now().strftime("%Y-%m-%d %H:%M:%S"),
        "text_cleaning_applied": True
    }
    
    # 처리된 데이터 저장
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(processed_data, f, ensure_ascii=False, indent=2)
    
    print(f"처리된 경제 뉴스 문서 수: {len(processed_data['documents'])}")
    print(f"데이터셋이 {output_path}에 저장되었습니다.")

In [17]:
# 경로 설정
news_train_path = "C:/Users/99kih/OneDrive/바탕 화면/논문 스터디/Project/Data/Training/신문기사_train_original"
news_val_path = "C:/Users/99kih/OneDrive/바탕 화면/논문 스터디/Project/Data/Validation/신문기사_valid_original"

output_train_path = "C:/Users/99kih/OneDrive/바탕 화면/논문 스터디/Project/Data/Training/processed_econ_news_train.json"
output_val_path = "C:/Users/99kih/OneDrive/바탕 화면/논문 스터디/Project/Data/Validation/processed_econ_news_val.json"

test_limit = 10  # 테스트용으로 10개 문서만 처리.

In [18]:
# 데이터셋 처리
process_news_dataset(news_train_path, output_train_path, limit=test_limit)

발견된 JSON 파일 수: 1


파일 처리 중:   0%|          | 0/1 [00:42<?, ?it/s]


처리된 경제 뉴스 문서 수: 10
데이터셋이 C:/Users/99kih/OneDrive/바탕 화면/논문 스터디/Project/Data/Training/processed_econ_news_train.json에 저장되었습니다.


In [19]:
process_news_dataset(news_val_path, output_val_path, limit=test_limit)

발견된 JSON 파일 수: 1


파일 처리 중:   0%|          | 0/1 [00:32<?, ?it/s]

처리된 경제 뉴스 문서 수: 10
데이터셋이 C:/Users/99kih/OneDrive/바탕 화면/논문 스터디/Project/Data/Validation/processed_econ_news_val.json에 저장되었습니다.
